<a href="https://colab.research.google.com/github/FlorentAndre/Florent_repository/blob/master/Geospatial_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

I will demonstrate my knowledge in Geospatial analysis by investigating the demographics of various counties in the state of California, and determine potentially suitable locations in San Francisco for 2 new Starbucks Reserve Roastery. 

In [2]:
pip install git+git://github.com/geopandas/geopandas.git

  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-28xh_635
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-28xh_635
     |████████████████████████████████| 14.8MB 306kB/s 
     |████████████████████████████████| 10.9MB 40.0MB/s 
  Created wheel for geopandas: filename=geopandas-0.8.0+37.gd693267-py2.py3-none-any.whl size=963512 sha256=96250d0e8cd13929c49b541ca99239805f9426e9bead6167a83d882377ac592b
  Stored in directory: /tmp/pip-ephem-wheel-cache-4ixeb1vc/wheels/91/24/71/376c9c67192694168352afcccc2d264248f7e2cc6192997186
Successfully built geopandas


In [3]:
# Installing the necessary libraries
! pip install folium
! pip install shapely
! pip install pygeos

     |████████████████████████████████| 1.6MB 3.3MB/s 


In [4]:
# Importing the necessary libraries
import math
import pygeos
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode 

import folium 
from folium import Choropleth, Marker
from folium.plugins import HeatMap, MarkerCluster

from shapely.geometry import MultiPolygon
from shapely.geometry import Point

/usr/local/lib/python3.6/dist-packages/geopandas/_compat.py:87: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


Using `embed_map()` function to visualize my maps.

In [5]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

## 1) Geo-localisation
I will create a dataframe `starbucks` containing Starbucks locations.

In [6]:
# Importing files from my Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Importing the Starbucks CSV
path_starbucks = "/content/drive/My Drive/Open Source data for Starbucks Geo Analytics/starbucks_locations.csv"
starbucks = pd.read_csv(path_starbucks)
# Dataset is now stored in a Pandas Dataframe
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


I will now check for any missing values, specifically Longitude and Latitude

In [8]:
# Creating a funciton to call for any df null rows which will allow me to check if any data is missing from my dataframe
def nans(df): return df[df.isnull().any(axis=1)]
# creating a df tp call for null rows - I will use the df later on.
rows_with_missing = nans(starbucks)
rows_with_missing

,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


### Conclusion on the missing rows.
 Only 5 rows have missing data and they are the Longitude and Latitue for the stores in Berkeley

### Using Geocode

I will create a funciton to to fill the missing values with the correct values from the OpenStreetMap Nominatim geocoder by using `geocode()`


In [9]:
def my_geocoder(row):
    try:
        point = geocode(row, provider='nominatim').geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None

berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


### 2) View Berkeley locations.
I will visualise the (latitude, longitude) locations in Berkeley in the OpenStreetMap style. 

In [27]:
# Create a base map
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=13)
# Add a marker for each Berkeley location
for idx, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)
# Show the map
m_2

### 3) Consolidate my data.

I will load a GeoDataFrame `CA_counties` containing the name, area (in square kilometers), and a unique id (in the "GEOID" column) for each county in the state of California.  The "geometry" column contains a polygon with county boundaries.

In [11]:
CA_counties = gpd.read_file("/content/drive/My Drive/Open Source data for Starbucks Geo Analytics/Shapefile/CA_county_boundaries.shp")
CA_counties.head()

,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


Next, I will create three DataFrames:
- `CA_pop` contains an estimate of the population of each county.
- `CA_high_earners` contains the number of households with an income of at least $150,000 per year.
- `CA_median_age` contains the median age for each county.

In [12]:
CA_pop = pd.read_csv("/content/drive/My Drive/Open Source data for Starbucks Geo Analytics/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("/content/drive/My Drive/Open Source data for Starbucks Geo Analytics/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("/content/drive/My Drive/Open Source data for Starbucks Geo Analytics/CA_county_median_age.csv", index_col="GEOID")

I will join `CA_counties` Geodataframe with `CA_pop`, `CA_high_earners`, and `CA_median_age` as a new Geodataframe called `CA_stats`. 
It should have 8 columns: and make sure it has 8 columns: "GEOID", "name", "area_sqkm", "geometry", "population", "high_earners", and "median_age".  I will also ensure it has a CRS set to `{'init': 'epsg:4326'}`.

In [13]:
# Joining all the following dataset's solumn into "cols_to_add"
cols_to_add = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
# Merging "cols_to_add" and "CA_stats" on GEOID
CA_stats = CA_counties.merge(cols_to_add, on="GEOID")

Now that we have all of the data in one place, it's much easier to calculate statistics than using a combination of columns.  
I will now create a "density" column with the population density.

In [14]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]

### 4) Which counties look promising?

I will look at a particular counties that matches the stakeholders requirements:

- there are at least 100,000 households making \$150,000 per year,
- the median age is less than 38.5, and
- the density of inhabitants is at least 285 (per square kilometer).

Additionally, selected counties should satisfy at least one of the following criteria:
- there are at least 500,000 households making \$150,000 per year,
- the median age is less than 35.5, or
- the density of inhabitants is at least 1400 (per square kilometer).

In [15]:
# My code here
sel_counties = CA_stats[((CA_stats.high_earners > 100000) &
                         (CA_stats.median_age < 38.5) &
                         (CA_stats.density > 285) &
                         ((CA_stats.median_age < 35.5) |
                         (CA_stats.density > 1400) |
                         (CA_stats.high_earners > 500000)))]

### 5) How many stores are identified?

When looking for the next Starbucks Reserve Roastery location, I'd like to consider all of the stores within the counties that I selected.  So, how many stores are within the selected counties?

To prepare to answer this question, I will create a GeoDataFrame `starbucks_gdf` with all of the starbucks locations.

In [16]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}
starbucks_gdf.head()

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,Store Number,Store Name,Address,City,Longitude,Latitude,geometry
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51,POINT (-117.40000 34.51000)
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16,POINT (-118.76000 34.16000)
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15,POINT (-118.76000 34.15000)
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87,POINT (-117.75000 33.87000)
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79,POINT (-122.28000 37.79000)


Finding out how many stores are in the counties selected?

In [17]:
# How many stores
locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
num_stores = len(locations_of_interest)
print("There are", num_stores, "stores in the counties that matches the Stakeholders requirements")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  


There are 1043 stores in the counties that matches the Stakeholders requirements


### 6) Visualize the store locations.

Create a map that shows the locations of the stores that I identified in the previous code.

In [28]:
# Create a base map
m_6 = folium.Map(location=[37,-120], zoom_start=6)
# Show selected store locations in a decluster way using MarkerCluster
mc = MarkerCluster()

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))
m_6.add_child(mc)
# Show the map
m_6

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  


### Looking where to build 2 new Stores in San Francisco

In [19]:
# Create "San_Francisco) df from "starbucks" 
San_Francisco = starbucks[starbucks["City"]=="San Francisco"]
# Creating a GeoDataFrame of "San_Francisco"
San_Francisco_gdf = gpd.GeoDataFrame(San_Francisco, geometry=gpd.points_from_xy(San_Francisco.Longitude, San_Francisco.Latitude))
# Adding the CRS data
San_Francisco_gdf.crs = {'init': 'epsg:4326'}
# Converting the ESPG to 2263 for unit purposes to convert in metres
San_Francisco_gdf = San_Francisco_gdf.to_crs(epsg=2263)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [20]:
# Creating a buffer of 1km radius to capture the coverage of each store in "San_Francisco"
buffer_in_meters = (1 * 1000) # Change the kms wished to cover
coverage = gpd.GeoDataFrame(geometry=San_Francisco_gdf.geometry).buffer(buffer_in_meters)

In [29]:
# Create map with release incidents and monitoring stations
m_7 = folium.Map(location=[37.77,-122.4], zoom_start=12)
# View SF stores
for idx, row in San_Francisco_gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_7)
# Plot each polygon on the map
folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m_7)
# Adding the Ltn and Lat data markers
folium.LatLngPopup().add_to(m_7)
# Show map - When clicking on the map, it will display the Lat. and Long.
m_7

In [30]:
# Provide 2 new locations for potential new stores
# Proposed location of hospital 1
lat_1 = 37.7771
long_1 = -122.4650

# Proposed location of hospital 2
lat_2 = 37.7442
long_2 = -122.4136

# Do not modify the code below this line - Creating a new_df based on choosen location
new_df = pd.DataFrame(
        {'Latitude': [lat_1, lat_2],
         'Longitude': [long_1, long_2]})
new_gdf = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df.Longitude, new_df.Latitude))
new_gdf.crs = {'init' :'epsg:4326'}
new_gdf = new_gdf.to_crs(epsg=2263) # Converting to ESPG 2263 to be on the same scale as previous map
# get new coverage
new_coverage = gpd.GeoDataFrame(geometry=new_gdf.geometry).buffer(buffer_in_meters)
# make the map
m_8 = folium.Map(location=[37.77,-122.4], zoom_start=12) 
folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m_8) # Adding existing coverage
folium.GeoJson(new_coverage.geometry.to_crs(epsg=4326)).add_to(m_8) # Adding new coverage
# Add new stores
for idx, row in new_gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']], icon=folium.Icon(color="red")).add_to(m_8)
# View existing stores
for idx, row in San_Francisco_gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_8)
folium.LatLngPopup().add_to(m_8)
# Display Map labelled with Red dots
m_8

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


## Conclusion
The above is where I believe we should build 2 new Starbucks Reserve Roastery. 